# CMU auto-graded notebook

Before you turn these assignments in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE`, `<FILL IN>`, or "YOUR ANSWER HERE."


---


# CMU Machine Learning with Large Datasets

## Homework 1 - Coding 2: Streaming Naive Bayes


In [0]:
# Who did you collaborate with on this assignment?
# if no one, collaborators should contain an empty string,
# else list your collaborators below

collaborators = []
print(collaborators)
# YOUR CODE HERE

[]


In [0]:
try:
    collaborators
except:
    raise AssertionError("you did not list your collaborators, if any")

### (1a) Environment Setup

Run the following cell to establish the runtime environment.


In [0]:
# Ignore this cell if the environment already has the packages
%pip install nose numpy

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
# imports that will be used in the notebook -- shouldn't need to import any other libraries

import math
import os
import re
from collections import Counter

import numpy as np
from nose.tools import assert_equal

### (1b) Data Preparation


We use the Reuters Corpus (RCV1), which is a set of news stories split into a hierarchy of categories. There are three file sets. The two data sets with "small" in the name contain smaller subsamples of the full data set. They are provided for debugging and local tests. The final classification task should use the "full" one. Each data set is split into a train and test set, as indicated by the file suffix:

- RCV1.full_train.txt
- RCV1.full_test.txt
- RCV1.small_train.txt
- RCV1.small_test.txt
- RCV1.very_small_train.txt
- RCV1.very_small_test.txt

Download the data using the link provided in the handout, and fill in the corresponding variables with their file paths in the following cell.


In [0]:
# Uncomment this cell if running on Databricks

from urllib.parse import urlparse
from pyspark import SparkFiles

def get_file(url):
    parsed_url = urlparse(url)
    suffix = parsed_url.path.split('/')[-1]
    sc.addFile(url)
    path = SparkFiles.get(suffix)

    return path

FULL_TRAIN = get_file("https://cmu.box.com/shared/static/g37w2q552qaik60yubrd3aq95szgi7pc")
FULL_TEST = get_file("https://cmu.box.com/shared/static/305174svzxe3lqm0ps76fvaenhr37inj")
SMALL_TRAIN = get_file("https://cmu.box.com/shared/static/f1x91jode5ywofu5b6cm9tb6ynx3yoft")
SMALL_TEST = get_file("https://cmu.box.com/shared/static/0l8y7x3gtix2sjfb1ewzq10c8ayaokdd")
VERY_SMALL_TRAIN = get_file("https://cmu.box.com/shared/static/xiumqmfdge5muxhs9v2w7ma97hurqtqy")
VERY_SMALL_TEST = get_file("https://cmu.box.com/shared/static/np2s7tn5wvwuawad2vddygs76tbwt1f5")

In [0]:
FULL_TRAIN = get_file("https://cmu.box.com/shared/static/g37w2q552qaik60yubrd3aq95szgi7pc")
FULL_TEST = get_file("https://cmu.box.com/shared/static/305174svzxe3lqm0ps76fvaenhr37inj")
SMALL_TRAIN = get_file("https://cmu.box.com/shared/static/f1x91jode5ywofu5b6cm9tb6ynx3yoft")
SMALL_TEST = get_file("https://cmu.box.com/shared/static/0l8y7x3gtix2sjfb1ewzq10c8ayaokdd")
VERY_SMALL_TRAIN = get_file("https://cmu.box.com/shared/static/xiumqmfdge5muxhs9v2w7ma97hurqtqy")
VERY_SMALL_TEST = get_file("https://cmu.box.com/shared/static/np2s7tn5wvwuawad2vddygs76tbwt1f5")

There are multiple class labels per document, meaning that there is more than one correct answer to the question "What kind of news article is this?"

For this assignment, we will ignore all class labels except for those ending in CAT and just be classifying into the top-level nodes of the hierarchy:

- CCAT: Corporate/Industrial
- ECAT: Economics
- GCAT: Government/Social
- MCAT: Markets

DO NOT change the following cell and just run it to set up the CAT labels


In [0]:
CAT_LABELS = ['CCAT', 'ECAT', 'GCAT', 'MCAT']

The data format is one document per line, with the class label(s) first (comma separated), a tab character, and then the document text.

Run the following cell to take a glance at the first document of the small training data set.


In [0]:
with open(SMALL_TRAIN, 'r') as f:
    print(f.readline())

C13,C21,CCAT	 The German government on Thursday awarded the first round of licences for basic public telephone services, opening the door for full competition to monopoly Deutsche Telekom. Kicking off the final stage of preparations ahead of liberalisation of European telecoms on January 1, 1998, Germany awarded licences for basic phone services to Vebacom GmbH, Britain's Colt Telecom Plc and NetCologne, a local operator.   The so-called &quot;class four&quot; licences cover basic voice telephone services for the public. That is the only service still under monopoly protection in Germany, where the telecoms market is expected to be worth more than 100 billion marks by the year 2000. In addition to the class 4 licences, the government awarded &quot;class 3&quot; licences to DBKom GmbH, the joint venture of a Mannesmann-led group and the German railway Deutsche Bahn AG, as well as to local operator VEW Telnet. Under a law passed this year, the class 3 licence allow owners of telecoms net

### (1c) Data Processing


To count the words, we need to tokenize the document text. In real-world practice, this may involve multiple steps, such as removing stop words and converting text to lowercase, which you learned in HW1: Entity Resolution. For this Naive Bayes part, we simplify the process by splitting only on words.

Run the following cell to define the `tokenization(doc)` function.


In [0]:
# DO NOT change this function
def tokenizeDoc(doc: str) -> list[str]:
    """
    Convert input document text into tokenization features
    Args:
        doc: document text
    Returns:
        list: a list of tokens
    """
    return re.findall('\\w+', doc)

As the handout instructs, streaming Naive Bayes loads one-line document at a time, use that document to update the classifier statistics, and then discard the document. After loading a line, we need a function to parse the line for classifier training.

Implement `parseDatafileLine(datafileLine)` that takes a line (labels + document text) and return a list of labels and a list of tokens. You need to use the `tokenizeDoc(doc)` function defined above.


In [0]:
def parseDatafileLine(datafileLine: str):
    parts = datafileLine.strip().split("\t", 1)  
    if len(parts) < 2:
        return [], []  
    labels_part, doc_text = parts  
    labels = labels_part.split(",")  
    tokens = tokenizeDoc(doc_text)  
    return labels, tokens


In [0]:
"""Test parseDatafileLine(datafileLine)"""

line_sample1 = "C15,C151,CCAT\tMcDonald's Corp said Thursday it raised its quarterly dividend 10 percent, to $0.0825 a share from $0.075."
line_sample2 = "C15,C151,CCAT\tSix months to Sept 30, 1996       (in million rupees unless stated)"

assert_equal(parseDatafileLine(line_sample1),
             (['C15', 'C151', 'CCAT'],
              ['McDonald', 's', 'Corp', 'said', 'Thursday', 'it', 'raised', 'its',
               'quarterly', 'dividend', '10', 'percent', 'to', '0', '0825', 'a',
               'share', 'from', '0', '075']))

assert_equal(parseDatafileLine(line_sample2),
             (['C15', 'C151', 'CCAT'],
              ['Six', 'months', 'to', 'Sept', '30', '1996', 'in', 'million',
               'rupees', 'unless', 'stated']))

### 1(d) Training


Now, we are good for training. We will use a dictionary as the model to store the count statistics.

As the handout instructs, the model contains five parts:

- `y`: $(Y=y)$ for each label y the number of training instances of that class
- `ys`: $(Y=*)$ here $*$ means anything, so this is just the total number of training instances
- `y_w`: $(Y=y,W=w)$ number of times token w appears in a document with label y
- `y_ws`: $(Y=y,W=*)$ total number of tokens for documents with label y
- `vocabulary`: track the vocabulary for Laplace smoothing

Recall that Laplace smoothing formula is

$$p(W=w_i|Y=y)=\frac{count(Y=y,W=w_i) + \alpha}{count(Y=y,W=*)+ \alpha|V|}$$
where $|V|$ is the vocabulary.

Implement `nbmodel()` by figuring out the proper variable types for each part and filling in the blank below.


In [0]:
def nbmodel() -> dict:
    """
    Initialize and return a dictionary to store Naive Bayes model statistics.

    Returns:
        dict storing the required five parts:
        - 'y': Dictionary storing count of training instances per class (Y=y).
        - 'ys': Integer representing the total number of training instances (Y=*).
        - 'y_w': Dictionary storing count of times each token appears in each class (Y=y, W=w).
        - 'y_ws': Dictionary storing the total count of words per class (Y=y, W=*).
        - 'vocabulary': Set of unique words encountered in the training data.
    """
    return {
        'y': {},          # Count of training instances per class {class_label: count}
        'ys': 0,          # Total number of training instances
        'y_w': {},        # Count of word occurrences per class {(class_label, word): count}
        'y_ws': {},       # Total word count per class {class_label: total_word_count}
        'vocabulary': set()  # Unique vocabulary for Laplace smoothing
    }


Implement `trainNB(trainfile, model)` that loads one document at a time and uses that document to update the required statistics for the Naive Bayes classifier.

Hint:

1. We only use those lables ending in CAT, which defined in `CAT_LABELS` earlier. Therefore, you need to figure out a way to skip other labels.
2. There are some documents with more than one CAT label. Treat those documents as multiple data instances (that is, add to the counters for
   all labels ending in CAT). For instance, if one line contains CCAT and ECAT, use the same document text twice.
3. It is not necessary to return the model here if you use proper types. Think about Python's mutable vs immutable types.


In [0]:
import math

def trainNB(trainfile: str, model: dict):
    """
    Train the Naive Bayes classifier by updating model statistics for each document.
    
    Args:
        trainfile (str): Path to the training data file.
        model (dict): Dictionary containing the Naive Bayes classifier model.
    """

    with open(trainfile, 'r', encoding="utf-8", errors="replace") as f:
        for line in f:
            labels, tokens = parseDatafileLine(line)

            model['ys'] += len(labels)

            for label in labels:
                if label.endswith("CAT"): 
                    
                    model['y'][label] = model['y'].get(label, 0) + 1  

                    if label not in model['y_ws']:
                        model['y_ws'][label] = 0  
                    unique_tokens = set(tokens)

                    for token in tokens:
                        model['y_w'][(label, token)] = model['y_w'].get((label, token), 0) + 1
                        model['y_ws'][label] += 1  # Increment total word count per class
                    
                    model['vocabulary'].update(unique_tokens)
    model['vocabulary_size'] = len(model['vocabulary'])


### 1(e) Test


For each line of documents, your classification code should get the best class $y$ and its log probabilities:

$$ln(p(Y=y))+\sum_{w_i} ln(p(W=w_i|Y=y))$$

Notice that we use the natural logarithm here.

Implement `testNB(testfile, model)` that uses the trained model to classify the test data and return a list of best classes, a list of max log probabilities (**rounding it to 4 decimal places**), and overall accuracy (**rounding it to 4 decimal places**). Please explicitly return in this specified order.


In [0]:
import math

def testNB(testfile, model):
    """
    Implement Naive Bayes classification.
    
    Args:
        testfile (str): File path of the test data.
        model (dict): The trained Naive Bayes model.
    
    Returns:
        best_classes (list): List of predicted labels for each document.
        log_probabilities (list): List of max log probabilities (rounded to 4 decimals).
        accuracy (float): Classification accuracy (rounded to 4 decimals).
    """
    best_classes = []
    log_probabilities = []
    correct_predictions = 0
    total_documents = 0

    class_counts = model['y']  
    total_docs = model['ys']  
    word_class_counts = model['y_w']  
    total_words_per_class = model['y_ws'] 
    vocab_size = len(model['vocabulary'])  
    alpha = 1  

    with open(testfile, 'r', encoding="utf-8", errors="replace") as f:
        for line in f:
            total_documents += 1
            true_labels, tokens = parseDatafileLine(line)

            log_prob_per_label = {}

            for label in class_counts:
                log_prob = math.log(class_counts[label] / total_docs)

                for token in tokens:
                    token_count = word_class_counts.get((label, token), 0)
                    total_tokens = total_words_per_class.get(label, 0) 
                    prob = (token_count + alpha) / (total_tokens + alpha * vocab_size)
                    log_prob += math.log(prob)

                log_prob_per_label[label] = log_prob

            best_label = max(log_prob_per_label, key=log_prob_per_label.get)

            max_log_prob = round(log_prob_per_label[best_label], 4)

            best_classes.append(best_label)
            log_probabilities.append(max_log_prob)

            if best_label in true_labels:
                correct_predictions += 1

    accuracy = round(correct_predictions / total_documents, 4)

    return best_classes, log_probabilities, accuracy


In [0]:
"""DO NOT change this this cell and just run it to use the very small dataset to test your implementations"""
very_small_model = nbmodel()
trainNB(VERY_SMALL_TRAIN, very_small_model)
best_classes, log_probabilities, accuracy = testNB(VERY_SMALL_TEST, very_small_model)
print(log_probabilities)

assert_equal(best_classes,
             ['MCAT', 'ECAT', 'CCAT', 'ECAT', 'CCAT', 'CCAT', 'ECAT', 'CCAT'])
assert_equal(accuracy, 0.8750)
assert_equal(log_probabilities,
             [-9893.7804, -3912.8180, -1121.5992, -1610.1660,
              -701.3466, -1453.3430, -2218.3302, -2285.0698])
assert_equal(accuracy, 0.8750)

[-9894.8165, -3913.8541, -1122.6353, -1611.2021, -702.3827, -1454.3791, -2219.3663, -2286.1059]


---------------------------------------------------------------------------
AssertionError                            Traceback (most recent call last)
File <command-2543043980408755>, line 10
      7 assert_equal(best_classes,
      8              ['MCAT', 'ECAT', 'CCAT', 'ECAT', 'CCAT', 'CCAT', 'ECAT', 'CCAT'])
      9 assert_equal(accuracy, 0.8750)
---> 10 assert_equal(log_probabilities,
     11              [-9893.7804, -3912.8180, -1121.5992, -1610.1660,
     12               -701.3466, -1453.3430, -2218.3302, -2285.0698])
     13 assert_equal(accuracy, 0.8750)

File /usr/lib/python3.10/unittest/case.py:845, in TestCase.assertEqual(self, first, second, msg)
    841 """Fail if the two objects are unequal as determined by the '=='
    842    operator.
    843 """
    844 assertion_func = self._getAssertEqualityFunc(first, second)
--> 845 assertion_func(first, second, msg=msg)

File /usr/lib/python3.10/unittest/case.py:1051, in TestCase.assertListEqual(self, list1, list2, msg)
   104

### 1(f) Full Classification and Deliverable

We are almost there! Let's wrap up this assignment.

Implement your training and test functions on the full dataset to get the full classification results. Please note that you need to define a new model different from the `very_small_model` we have already tested.

Write the full classification results to a file `full_result.txt` (please explicitly use this name). The output format should have one test result per line, and each line should have the format:

$$\text{[Label1, Label2, ...]<tab>Best class<tab>Log prob}$$

where **[Label1, Label2, ...]** are the true labels of the test instance, **Best class** is the class with the maximum log probability, and the last field is the log probability.

The last line of the file should include the accuracy.

Use the following cell to write your code.

Submit both this notebook and `full_result.txt` to Gradescope.


In [0]:
import os

def writeFullResults(trainfile: str, testfile: str, outputfile: str):
    if not os.path.exists(testfile):
        raise FileNotFoundError(f"❌ Test file not found: {testfile}")
    if not os.path.exists(trainfile):
        raise FileNotFoundError(f"❌ Training file not found: {trainfile}")

    full_model = {
        'y': {},  # Label counts
        'ys': 0,  # Total number of training instances
        'y_w': {},  # (Y=y, W=w): Count of words per label
        'y_ws': {},  # (Y=y, W=*): Total word count per label
        'vocabulary': set()
    }

    trainNB(trainfile, full_model)

    best_classes, log_probabilities, accuracy = testNB(testfile, full_model)

    print(f"Opening test file: {testfile}")

    try:
        with open(testfile, 'r', encoding='utf-8', errors='replace') as test_f, \
             open(outputfile, 'w', encoding='utf-8') as out_f:
            
            for i, line in enumerate(test_f):
                true_labels, _ = parseDatafileLine(line)
                out_f.write(f"{true_labels}\t{best_classes[i]}\t{log_probabilities[i]:.4f}\n")

            out_f.write(f"Accuracy: {accuracy:.4f}\n")

    except UnicodeDecodeError:
        print("⚠️ UnicodeDecodeError: Retrying with 'latin1' encoding...")
        with open(testfile, 'r', encoding='latin1') as test_f, \
             open(outputfile, 'w', encoding='utf-8') as out_f:

            for i, line in enumerate(test_f):
                true_labels, _ = parseDatafileLine(line)
                out_f.write(f"{true_labels}\t{best_classes[i]}\t{log_probabilities[i]:.4f}\n")

            out_f.write(f"Accuracy: {accuracy:.4f}\n")

writeFullResults(FULL_TRAIN, FULL_TEST, "full_result.txt")
print("✅ Classification results saved to 'full_result.txt'")


---------------------------------------------------------------------------
AssertionError                            Traceback (most recent call last)
File <command-2543043980408755>, line 10
      7 assert_equal(best_classes,
      8              ['MCAT', 'ECAT', 'CCAT', 'ECAT', 'CCAT', 'CCAT', 'ECAT', 'CCAT'])
      9 assert_equal(accuracy, 0.8750)
---> 10 assert_equal(log_probabilities,
     11              [-9893.7804, -3912.8180, -1121.5992, -1610.1660,
     12               -701.3466, -1453.3430, -2218.3302, -2285.0698])
     13 assert_equal(accuracy, 0.8750)

File /usr/lib/python3.10/unittest/case.py:845, in TestCase.assertEqual(self, first, second, msg)
    841 """Fail if the two objects are unequal as determined by the '=='
    842    operator.
    843 """
    844 assertion_func = self._getAssertEqualityFunc(first, second)
--> 845 assertion_func(first, second, msg=msg)

File /usr/lib/python3.10/unittest/case.py:1051, in TestCase.assertListEqual(self, list1, list2, msg)
   104